In [1]:
import torch
import torch.nn as nn
import egg.core as core

from torchvision import datasets, transforms
from torch import nn
from torch.nn import functional as F
from torch.distributions import Categorical
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import random
import numpy as np

import csv
import re
import os
import unicodedata
import codecs
import itertools
from xml.dom import minidom
import xml.etree.ElementTree as ET


from pylab import rcParams
rcParams['figure.figsize'] = 5, 10

# For convenince and reproducibility, we set some EGG-level command line arguments here
opts = core.init(params=['--random_seed=7', # will initialize numpy, torch, and python RNGs
                         '--lr=1e-3',   # sets the learning rate for the selected optimizer 
                         '--batch_size=32',
                         '--optimizer=adam'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(0)
np.random.seed(0)

In [2]:
class Vocabulary:
    def __init__(self):
        self.attribute2index = {}
        self.attribute2count = {}
        self.index2attribute = {}
        self.num_attributes = 0
        self.concept_list = []
        self.concept2vector = {}
        self.num_concepts = 0
        
    def schredderConcept(self, animal):
        i = 0
        attributes= []
        for node in animal.childNodes:
            i+=1
            if i%2 == 0:
                for attribute in node.childNodes[0].data.split('\n'):
                    if attribute != '':
                        attributes.append(attribute.strip())
        return attributes
        
    def addConcept(self, animal):
        self.num_concepts += 1
        self.concept_list.append(animal)
        attributes = self.schredderConcept(animal)
        for attribute in attributes:
            self.addAttr(attribute)                        
                        
            
    def addAttr(self,attribute):
        if attribute not in self.attribute2index:
            self.attribute2index[attribute] = self.num_attributes
            self.attribute2count[attribute] = 1
            self.index2attribute[self.num_attributes] = attribute
            self.num_attributes += 1
        else:
            self.attribute2count[attribute] += 1
            
    def parseConcept(self, animal):
        if animal.attributes['name'].value not in self.concept2vector:
            attributes = self.schredderConcept(animal)
            vector = torch.zeros(self.num_attributes)
            for attribute in attributes:
                vector[self.attribute2index[attribute]]=1
            self.concept2vector[animal.attributes['name'].value] = vector
        else:
            vector = self.concept2vector[animal.attributes['name'].value]
        return vector
    
    def addFile(self, name):
        file_name = name + "_structured_final.us.xml"
        file = minidom.parse(os.path.join(os.path.join("visa_dataset", "US"), file_name))
        concepts = file.getElementsByTagName('concept')

        for concept in concepts:
            self.addConcept(concept)

In [3]:
class ConceptDataSet(Dataset):
    def __init__(self, voc, num_options):
        super(ConceptDataSet, self).__init__()
        self.voc = voc
        self.num_options = num_options
        
    def __len__(self):
        return self.voc.num_concepts
    
    def __getitem__(self, idx):
        concept = self.voc.concept_list[idx]
        sender_input = self.voc.parseConcept(concept)
        
        commute = torch.randint(0,self.num_options,[1])
        
        receiver_input = []
        for option in range(self.num_options):
            if option==commute:
                receiver_input.append(sender_input)
            else:
                x = list(range(1,idx)) + list(range(idx+1, self.voc.num_concepts))
                r = random.choice(x)
                c = self.voc.concept_list[r]
                receiver_input.append(self.voc.parseConcept(c))      
        sample = [sender_input, commute, receiver_input]
        return sample

In [4]:
concept_list = Vocabulary()

concept_list.addFile("ANIMALS")
concept_list.addFile("APPLIANCES")
concept_list.addFile("ARTIFACTS")
concept_list.addFile("CLOTHING")
concept_list.addFile("CONTAINER")
concept_list.addFile("DEVICE")
concept_list.addFile("FOOD")
concept_list.addFile("HOME")
concept_list.addFile("INSTRUMENTS")
concept_list.addFile("MATERIAL")
concept_list.addFile("PLANTS")
concept_list.addFile("STRUCTURES")
concept_list.addFile("TOOLS")
concept_list.addFile("TOYS")
concept_list.addFile("VEHICLES")
concept_list.addFile("WEAPONS")

print(concept_list.num_concepts)
print(concept_list.num_attributes)

509
597


In [5]:
vocab_size = 100
max_len = 5
num_options = 5
batch_size = 32

In [6]:
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

train_loader = torch.utils.data.DataLoader(
        ConceptDataSet(concept_list, num_options),
           batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
        ConceptDataSet(concept_list, num_options),
           batch_size=batch_size, shuffle=False, **kwargs)

In [7]:
class PrePro(nn.Module):
    def __init__(self, hidden_size, voc):
        super(PrePro, self).__init__()
        self.pre_linguistic = nn.Linear(voc.num_attributes, hidden_size)
        self.act = nn.Sigmoid()
        
    def forward(self, input_concept):
        raw = self.pre_linguistic(input_concept)
        output = self.act(raw)
        return output
    
    
class Receiver(nn.Module):
    def __init__(self, hidden_size, num_options, voc):
        super(Receiver, self).__init__()
        self.encoder = PrePro(hidden_size,voc)
        self.num_options = num_options
        
    def forward(self, channel_input, receiver_input=None):
        c_list =  []
        for i in range(self.num_options):
            c_list.append(self.encoder(receiver_input[i]))
        concepts = torch.stack(c_list)  
        concepts= concepts.transpose(0,1)

        channel_input = channel_input[:,:,None]

        dotproduct = torch.bmm(concepts, channel_input)
        dotproduct = dotproduct.squeeze(dim=-1)

        pre_logits = F.log_softmax(dotproduct, dim=1)

        distr = Categorical(logits=pre_logits)

        if self.training:
            sample = distr.sample()
        else:
            sample = pre_logits.argmax(dim=1)
        log_prob = distr.log_prob(sample)

        entropy = distr.entropy()

        return sample, log_prob, entropy

In [8]:
#loss um prediction mit Realität zu vergleichen
def loss(_sender_input, _message, _receiver_input, receiver_output, labels):
    acc = (labels.t() == receiver_output).float() - (labels.t() != receiver_output).float()
    return -acc, {'acc': (acc.mean().item()+1)/2}

In [9]:
# we start by defining the RNN parameters
hidden_size = 50
emb_size = 50

In [10]:
sender = PrePro(hidden_size, concept_list)
receiver = Receiver(hidden_size, num_options, concept_list)

sender_rnn = core.RnnSenderReinforce(sender, vocab_size, emb_size, hidden_size,
                                   cell="gru", max_len= max_len)
receiver_rnn = core.RnnReceiverReinforce(receiver, vocab_size, emb_size,
                    hidden_size, cell="gru")

game_rnn = core.SenderReceiverRnnReinforce(sender_rnn, receiver_rnn, loss, 
                                           sender_entropy_coeff=0.015, 
                                           receiver_entropy_coeff=0.0)

In [11]:
optimizer = torch.optim.Adam([
        {'params': game_rnn.sender.parameters(), 'lr': 1e-3},
        {'params': game_rnn.receiver.parameters(), 'lr': 1e-2}
    ])

In [12]:
trainer = core.Trainer(game=game_rnn, optimizer=optimizer, train_data=train_loader,
                           validation_data=test_loader)
common_opts = core.util.get_opts()

torch.manual_seed(0)
np.random.seed(0)

trainer.train(2000)

test: epoch 1, loss 1.1651580333709717,  {'acc': 0.20332704670727253, 'loss': 1.1651578396558762, 'sender_entropy': 3.668747663497925, 'receiver_entropy': 0.8644513785839081, 'original_loss': 0.5933459065854549, 'mean_length': 5.0}
test: epoch 2, loss 0.19204005599021912,  {'acc': 0.189453125, 'loss': 0.19203984178602695, 'sender_entropy': 3.669870764017105, 'receiver_entropy': 1.183212399482727, 'original_loss': 0.62109375, 'mean_length': 5.0}
test: epoch 3, loss 1.087148666381836,  {'acc': 0.22117456886917353, 'loss': 1.0871484503149986, 'sender_entropy': 3.668641835451126, 'receiver_entropy': 1.2828617170453072, 'original_loss': 0.557650862261653, 'mean_length': 5.0}
test: epoch 4, loss 0.639578104019165,  {'acc': 0.20918642170727253, 'loss': 0.639578215777874, 'sender_entropy': 3.6684793531894684, 'receiver_entropy': 0.8030504770576954, 'original_loss': 0.5816271565854549, 'mean_length': 5.0}
test: epoch 5, loss 0.15696567296981812,  {'acc': 0.189453125, 'loss': 0.15696591511368752

test: epoch 37, loss 0.6082987189292908,  {'acc': 0.19005926698446274, 'loss': 0.6082987282425165, 'sender_entropy': 3.644074723124504, 'receiver_entropy': 0.4760678540915251, 'original_loss': 0.6198814660310745, 'mean_length': 2.7667025923728943}
test: epoch 38, loss 0.33405113220214844,  {'acc': 0.1879040952771902, 'loss': 0.33405131101608276, 'sender_entropy': 3.647315338253975, 'receiver_entropy': 0.3693010676652193, 'original_loss': 0.6241918094456196, 'mean_length': 3.044921875}
test: epoch 39, loss 1.1116092205047607,  {'acc': 0.2250134702771902, 'loss': 1.1116090528666973, 'sender_entropy': 3.6457328647375107, 'receiver_entropy': 0.45990858413279057, 'original_loss': 0.5499730594456196, 'mean_length': 3.01953125}
test: epoch 40, loss 0.7888239622116089,  {'acc': 0.20763739198446274, 'loss': 0.7888238932937384, 'sender_entropy': 3.644402340054512, 'receiver_entropy': 0.3331670146435499, 'original_loss': 0.5847252160310745, 'mean_length': 4.994140625}
test: epoch 41, loss 0.67216

test: epoch 72, loss 0.90512615442276,  {'acc': 0.2132947202771902, 'loss': 0.9051264617592096, 'sender_entropy': 3.641399011015892, 'receiver_entropy': 0.3042495120316744, 'original_loss': 0.5734105594456196, 'mean_length': 5.0}
test: epoch 73, loss 0.7699897289276123,  {'acc': 0.2074353452771902, 'loss': 0.7699892893433571, 'sender_entropy': 3.6369826197624207, 'receiver_entropy': 0.2990214051678777, 'original_loss': 0.5851293094456196, 'mean_length': 5.0}
test: epoch 74, loss 0.6897301077842712,  {'acc': 0.20332704670727253, 'loss': 0.6897302307188511, 'sender_entropy': 3.633037820458412, 'receiver_entropy': 0.3105747355148196, 'original_loss': 0.5933459065854549, 'mean_length': 5.0}
test: epoch 75, loss 0.6648117899894714,  {'acc': 0.20137392170727253, 'loss': 0.6648115068674088, 'sender_entropy': 3.606956735253334, 'receiver_entropy': 0.31795430835336447, 'original_loss': 0.5972521565854549, 'mean_length': 5.0}
test: epoch 76, loss 0.6420835852622986,  {'acc': 0.20022898726165295,

test: epoch 108, loss -0.2086695432662964,  {'acc': 0.173828125, 'loss': -0.20866940915584564, 'sender_entropy': 3.6085041165351868, 'receiver_entropy': 0.29302372224628925, 'original_loss': 0.65234375, 'mean_length': 5.0}
test: epoch 109, loss 0.27579808235168457,  {'acc': 0.19160829670727253, 'loss': 0.2757979352027178, 'sender_entropy': 3.612150326371193, 'receiver_entropy': 0.29665561206638813, 'original_loss': 0.6167834065854549, 'mean_length': 5.0}
test: epoch 110, loss 1.0661661624908447,  {'acc': 0.22090517170727253, 'loss': 1.06616622954607, 'sender_entropy': 3.6108845472335815, 'receiver_entropy': 0.27481463085860014, 'original_loss': 0.5581896565854549, 'mean_length': 5.0}
test: epoch 111, loss 0.40173017978668213,  {'acc': 0.1957165952771902, 'loss': 0.40173017606139183, 'sender_entropy': 3.6088587194681168, 'receiver_entropy': 0.2891783704981208, 'original_loss': 0.6085668094456196, 'mean_length': 5.0}
test: epoch 112, loss 0.560309648513794,  {'acc': 0.201171875, 'loss': 

test: epoch 144, loss 0.4717465043067932,  {'acc': 0.1976697202771902, 'loss': 0.4717465750873089, 'sender_entropy': 3.4291610717773438, 'receiver_entropy': 0.27633725106716156, 'original_loss': 0.6046605594456196, 'mean_length': 5.0}
test: epoch 145, loss 0.668145477771759,  {'acc': 0.2093884702771902, 'loss': 0.6681452933698893, 'sender_entropy': 3.4177893847227097, 'receiver_entropy': 0.2732151160016656, 'original_loss': 0.5812230594456196, 'mean_length': 5.0}
test: epoch 146, loss 0.1836506575345993,  {'acc': 0.1839978452771902, 'loss': 0.1836506500840187, 'sender_entropy': 3.424851879477501, 'receiver_entropy': 0.3009080486372113, 'original_loss': 0.6320043094456196, 'mean_length': 5.0}
test: epoch 147, loss -0.11663314700126648,  {'acc': 0.19551454670727253, 'loss': -0.11663307249546051, 'sender_entropy': 3.517250820994377, 'receiver_entropy': 0.267295247875154, 'original_loss': 0.6089709065854549, 'mean_length': 5.0}
test: epoch 148, loss 0.489516019821167,  {'acc': 0.2015759702

...

test: epoch 1829, loss -0.5054405927658081,  {'acc': 0.7774784481152892, 'loss': -0.505440654233098, 'sender_entropy': 0.08632374536682619, 'receiver_entropy': 0.3477106876671314, 'original_loss': -0.5549568962305784, 'mean_length': 5.0}
test: epoch 1830, loss -0.5053991675376892,  {'acc': 0.7863011844456196, 'loss': -0.5053991936147213, 'sender_entropy': 0.08439853730669711, 'receiver_entropy': 0.33860490284860134, 'original_loss': -0.5726023688912392, 'mean_length': 5.0}
test: epoch 1831, loss -0.47694700956344604,  {'acc': 0.745083512738347, 'loss': -0.47694696206599474, 'sender_entropy': 0.0859073486062698, 'receiver_entropy': 0.3593873009085655, 'original_loss': -0.4901670254766941, 'mean_length': 5.0}
test: epoch 1832, loss -0.49834081530570984,  {'acc': 0.7683189660310745, 'loss': -0.4983408283442259, 'sender_entropy': 0.08194511464535026, 'receiver_entropy': 0.3335794676095247, 'original_loss': -0.536637932062149, 'mean_length': 5.0}
test: epoch 1833, loss -0.5030481219291687, 

test: epoch 1864, loss -0.4941898584365845,  {'acc': 0.7823949344456196, 'loss': -0.4941898249089718, 'sender_entropy': 0.09978567982034292, 'receiver_entropy': 0.32823990657925606, 'original_loss': -0.5647898688912392, 'mean_length': 5.0}
test: epoch 1865, loss -0.48787757754325867,  {'acc': 0.7747844830155373, 'loss': -0.48787759244441986, 'sender_entropy': 0.09124167630216107, 'receiver_entropy': 0.3174579767510295, 'original_loss': -0.5495689660310745, 'mean_length': 5.0}
test: epoch 1866, loss -0.5021611452102661,  {'acc': 0.7858970910310745, 'loss': -0.5021611265838146, 'sender_entropy': 0.0915274372237036, 'receiver_entropy': 0.30622496642172337, 'original_loss': -0.571794182062149, 'mean_length': 5.0}
test: epoch 1867, loss -0.47223225235939026,  {'acc': 0.7858970910310745, 'loss': -0.4722321992740035, 'sender_entropy': 0.0954554239870049, 'receiver_entropy': 0.3276836620643735, 'original_loss': -0.571794182062149, 'mean_length': 5.0}
test: epoch 1868, loss -0.488597571849823, 

test: epoch 1899, loss -0.48023363947868347,  {'acc': 0.7630657330155373, 'loss': -0.48023364786058664, 'sender_entropy': 0.09726025335112354, 'receiver_entropy': 0.3572667930275202, 'original_loss': -0.5261314660310745, 'mean_length': 5.0}
test: epoch 1900, loss -0.4741669297218323,  {'acc': 0.7591594830155373, 'loss': -0.474166925996542, 'sender_entropy': 0.09888833466538927, 'receiver_entropy': 0.3479413893073797, 'original_loss': -0.5183189660310745, 'mean_length': 5.0}
test: epoch 1901, loss -0.4874454736709595,  {'acc': 0.7720231674611568, 'loss': -0.4874454867094755, 'sender_entropy': 0.09954114324500551, 'receiver_entropy': 0.35053940303623676, 'original_loss': -0.5440463349223137, 'mean_length': 5.0}
test: epoch 1902, loss -0.4710739254951477,  {'acc': 0.7728313580155373, 'loss': -0.4710738807916641, 'sender_entropy': 0.09451219086622586, 'receiver_entropy': 0.37783046904951334, 'original_loss': -0.5456627160310745, 'mean_length': 5.0}
test: epoch 1903, loss -0.495932221412658

test: epoch 1934, loss -0.4996141195297241,  {'acc': 0.7765355594456196, 'loss': -0.4996141195297241, 'sender_entropy': 0.09440830958192237, 'receiver_entropy': 0.3534619715064764, 'original_loss': -0.5530711188912392, 'mean_length': 5.0}
test: epoch 1935, loss -0.5335121750831604,  {'acc': 0.7999730594456196, 'loss': -0.5335121974349022, 'sender_entropy': 0.08347737629082985, 'receiver_entropy': 0.3124608267098665, 'original_loss': -0.5999461188912392, 'mean_length': 5.0}
test: epoch 1936, loss -0.5165717601776123,  {'acc': 0.7858970910310745, 'loss': -0.5165717639029026, 'sender_entropy': 0.0912077283137478, 'receiver_entropy': 0.3457091171294451, 'original_loss': -0.571794182062149, 'mean_length': 5.0}
test: epoch 1937, loss -0.5165058374404907,  {'acc': 0.7908135782927275, 'loss': -0.5165058430284262, 'sender_entropy': 0.09797962449374609, 'receiver_entropy': 0.3288180623203516, 'original_loss': -0.5816271565854549, 'mean_length': 5.0}
test: epoch 1938, loss -0.5254138112068176,  {

test: epoch 1969, loss -0.5224943161010742,  {'acc': 0.7804418094456196, 'loss': -0.5224942695349455, 'sender_entropy': 0.0752454706816934, 'receiver_entropy': 0.3248773254454136, 'original_loss': -0.5608836188912392, 'mean_length': 5.0}
test: epoch 1970, loss -0.4984908401966095,  {'acc': 0.776333512738347, 'loss': -0.49849086068570614, 'sender_entropy': 0.08117486625269521, 'receiver_entropy': 0.34506074991077185, 'original_loss': -0.5526670254766941, 'mean_length': 5.0}
test: epoch 1971, loss -0.5081911087036133,  {'acc': 0.7747844830155373, 'loss': -0.5081911031156778, 'sender_entropy': 0.0840686834853841, 'receiver_entropy': 0.35440665669739246, 'original_loss': -0.5495689660310745, 'mean_length': 5.0}
test: epoch 1972, loss -0.5247041583061218,  {'acc': 0.7839439660310745, 'loss': -0.5247040949761868, 'sender_entropy': 0.07356513827107847, 'receiver_entropy': 0.3389818677678704, 'original_loss': -0.567887932062149, 'mean_length': 5.0}
test: epoch 1973, loss -0.4981498420238495,  